## Scrapper de Leilões de carros
### Coisas importantes:
1. Modelo do carro
2. Ano do carro
3. Sinistro
4. Quilometragem
5. Data do leilão
6. Link para o carro

### Segunda etapa
1. Pelo link pegar o preco do maior lance
2. incremento min

Entra no link, pega todas informações de cada carro
passa pra proxima página, até a ultima
se achar algum que bate com o nome colocado de entrada
ele separa as informações e envia via WhatsApp

In [78]:
#-- IMPORT LIBS
from re import search
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromiumService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from webdriver_manager.core.utils import ChromeType
from datetime import datetime
from pywhatkit import sendwhatmsg
import csv
from time import sleep

def get_time() -> tuple:
    """
    Funcao para pegar a hora
    do local em que voce esta
    :return:
    """
    currentDateAndTime = datetime.now()

    currentTime_hours = currentDateAndTime.strftime("%H")
    currentTime_minutes = currentDateAndTime.strftime("%M")

    return currentTime_hours, currentTime_minutes

def navegador() -> any:
    """
    Funcao para iniciar o navegador chrome
    :param start_url: url para abrir o navegador
    :return:
    """
    #-- Opcoes para iniciar driver selenium
    options = webdriver.ChromeOptions()
    # options.add_argument('--headless')
    options.add_argument('--ignore-certificate-errors')

    #-- Instala o chromedriver e retorna o obj do selenium
    chromium = ChromeDriverManager(chrome_type = ChromeType.CHROMIUM)
    driver = webdriver.Chrome(service = ChromiumService((chromium).install()), options=options)

    return driver

def wpp(celular:str, msg:str) -> None:
    """
    Funcao para enviar msg no wpp,
    Quando for colocar o numero precisa do codigo
    do pais +55
    :param celular: numero de celular que deseja enviar a msg
    :param msg: mensagem para enviar via WPP
    :return:
    """
    print('Em alguns segundos WhatsApp vai abrir e em 15 segundos enviar sua msg!')
    hora, minuto = get_time()
    sendwhatmsg(celular, msg, int(hora), int(minuto)+2, tab_close=True)

    return

def create_log() -> None:
    """
    Funcao para criar arquivo de logs
    :return:
    """
    try:
        with open('/logs.csv', 'x') as f:
            writer = csv.DictWriter(f, fieldnames = ["nome_carro", "modelo", "versao", "data_leilao",
                                                      "hora_leilao", "preco_inicial", "ano_carro",
                                                      "lance_atual", "link"])
            writer.writeheader()

        print('Arquivo de logs criado com sucesso')
    except:
        print('Arquivo de logs ja criado')

    return

create_log()

def log(dictionary:dict) -> None:
    """
    Funcao para adicionar arquivo de logs,
    para nao repetir carros ja enviados
    :param dictionary: insere valores encontrados no arquivo de logs
    :return:
    """
    fieldnames = ["nome_carro", "modelo", "versao", "data_leilao",
                  "hora_leilao", "preco_inicial", "ano_carro",
                  "lance_atual", "link"]

    with open('/logs.csv', 'a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writerow(dictionary)

    return

def get_num_pages():
    """
    Funcao para pegar o numero total de paginas
    :return:
    """
    driver = navegador()
    driver.get('https://www.freitasleiloeiro.com.br/leiloes/pesquisar?pg=100&categoria=1')

    num_page = driver.find_element(By.CLASS_NAME, 'pagination').text
    numero_paginas = int(num_page[-2:])
    return numero_paginas

def scrapper(modelo_carro:str) -> list:
    """
    Scrapper para comecar a verificar os sites,
    Use o modelo do carro em letras maiusculas
    :param modelo_carro: Inserir modelo em letras maiusculas
    :return:
    """
    total_pag = get_num_pages()
    print(f'Temos {total_pag} paginas de carros')

    driver = navegador()

    cars_found = list()
    for pag in range(1, total_pag - 1):

        sleep(1)

        print(f'Pegando as informacoes da pagina {pag}...')

        driver.get(f'https://www.freitasleiloeiro.com.br/leiloes/pesquisar?pg={pag}&categoria=1')

        # Driver espera ate encontrar o elemento body
        WebDriverWait(driver, timeout=10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))

        # Procura as descricoes dos carros
        carros = driver.find_elements(By.CLASS_NAME, 'cursor-pointer ')

        # Patterns para o regex search
        nome_carro = f'{modelo_carro}'
        pattern_model = f'(?={modelo_carro}).+?(?<=,)'
        pattern_link_model = r"(?=').+(?<=')"
        pattern_data = r"[0-9]{2}\/[0-9]{2}\/[0-9]{4}"
        pattern_hora_leilao = r'[0-9]{2}:[0-9]{2}'
        patterno_preco_inicial = r'(?<=R\$ ).+'
        pattern_ano_carro = r"(?<=,).+?(?<=,)"
        pattern_lance_atual = r"R\$\s*([\d\.,]+)"
        aspas = r"'"
        vazio = ""

        for i in carros:
            if search(nome_carro, i.text):
                # Separa as info do carro que foi encontrado
                link          = search(pattern_link_model, i.get_attribute("onclick")).group(0)
                link_carro    = f"https://www.freitasleiloeiro.com.br{link.replace(aspas, vazio)}"
                modelo        = search(pattern_model, i.text).group(0)
                data_leilao   = search(pattern_data, i.text).group(0)
                hora_leilao   = search(pattern_hora_leilao, i.text).group(0)
                preco_inicial = search(patterno_preco_inicial, i.text).group(0)
                ano_carro     = search(pattern_ano_carro, i.text).group(0)

                # Entra no link do carro que achou e pega o lance atual
                driver.get(link_carro)
                try:
                    str_lance_atual = driver.find_element(By.ID, 'lance-atual').text
                    lance_atual     = search(pattern_lance_atual, str_lance_atual).group(0)
                except:
                    lance_atual = 'Sem lances'

                sleep(1)
                # Driver volta para pagina anterior para continuar
                driver.execute_script("window.history.go(-1)")
                WebDriverWait(driver, timeout=10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))

                car = {
                    "nome_carro": nome_carro,
                    "modelo": modelo,
                    "data_leilao": data_leilao,
                    "hora_leilao": hora_leilao,
                    "preco_inicial": preco_inicial,
                    "ano_carro": ano_carro,
                    "lance_atual": lance_atual,
                    "link": link_carro}

                # Insere o log do carro que achou
                log(car)
                # Adiciona o carro pra lista de carros encontrados
                cars_found.append(car)
        WebDriverWait(driver, timeout=10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    driver.quit()

    return cars_found

def filtra_carros_encontrados(cars_found:list) -> list:
    """
    Funcao para verificar se os carros encontrados pelo scrapper
    ja estao no log, se nao estiverem, envia msg
    :param cars_found: lista de dicionario dos carros encontrados
    :return:
    """
    with open('/logs.csv', 'r') as f:
        log = csv.DictReader(f)

        # Cria uma lista com os links que o scrapper pegou
        link_values = [d['link'] for d in cars_found if 'link' in d]

        # Loop para tirar os carros que ja foram enviados, olhando o link
        for link in link_values:
            for row in log:
                if row['link'] == link:
                    del cars_found[link]
                    break
                if not row['link'] == link:
                    return

    return cars_found

def envia_msg(list_dict_carros:list) -> None:
    """
    Funcao para enviar a msg filtrada no wpp
    :param list_dict_carros: lista de carros filtrados
    :return:
    """
    for carro in list_dict_carros:

        mensagem = f"Oportunidade!\n{carro.get('modelo')} {carro.get('ano_carro')}\nPreco inicial: {carro.get('preco_inicial')}\n{carro.get('lance_atual')}\nLeilao no dia: {carro.get('data_leilao')} as {carro.get('hora_leilao')}\nConfira o carro no link: {carro.get('link_carro')}"

        wpp("+55", mensagem)
    return

Arquivo de logs ja criado


### Alguns testes que estou realizando enquanto faco o codigo principal estarao abaixo

In [74]:
carros_encontrados2 = scrapper('ONIX')

Temos 26 paginas de carros
Pegando as informacoes da pagina 1...
Pegando as informacoes da pagina 2...
Pegando as informacoes da pagina 3...
Pegando as informacoes da pagina 4...
Pegando as informacoes da pagina 5...
Pegando as informacoes da pagina 6...
Pegando as informacoes da pagina 7...
Pegando as informacoes da pagina 8...
Pegando as informacoes da pagina 9...
Pegando as informacoes da pagina 10...
Pegando as informacoes da pagina 11...
Pegando as informacoes da pagina 12...
Pegando as informacoes da pagina 13...
Pegando as informacoes da pagina 14...
Pegando as informacoes da pagina 15...
Pegando as informacoes da pagina 16...
Pegando as informacoes da pagina 17...
Pegando as informacoes da pagina 18...
Pegando as informacoes da pagina 19...
Pegando as informacoes da pagina 20...
Pegando as informacoes da pagina 21...
Pegando as informacoes da pagina 22...
Pegando as informacoes da pagina 23...
Pegando as informacoes da pagina 24...


In [75]:
carros_encontrados2

[{'nome_carro': 'ONIX',
  'modelo': 'ONIX 10MT JOYE,',
  'data_leilao': '15/03/2023',
  'hora_leilao': '10:00',
  'preco_inicial': '25.000,00',
  'ano_carro': ' 19/19,',
  'lance_atual': 'R$ 25.000,00',
  'link': 'https://www.freitasleiloeiro.com.br/leiloes/lote?leilaoid=5767&lote=8'},
 {'nome_carro': 'ONIX',
  'modelo': 'ONIX 1.0MT LT,',
  'data_leilao': '15/03/2023',
  'hora_leilao': '10:00',
  'preco_inicial': '19.000,00',
  'ano_carro': ' 13/13,',
  'lance_atual': 'R$ 22.500,00',
  'link': 'https://www.freitasleiloeiro.com.br/leiloes/lote?leilaoid=5767&lote=99'},
 {'nome_carro': 'ONIX',
  'modelo': 'ONIX 1.0MT LT,',
  'data_leilao': '15/03/2023',
  'hora_leilao': '10:00',
  'preco_inicial': '19.000,00',
  'ano_carro': ' 15/16,',
  'lance_atual': 'R$ 20.000,00',
  'link': 'https://www.freitasleiloeiro.com.br/leiloes/lote?leilaoid=5767&lote=213'},
 {'nome_carro': 'ONIX',
  'modelo': 'ONIX 1.0MT LT,',
  'data_leilao': '15/03/2023',
  'hora_leilao': '10:00',
  'preco_inicial': '16.000,